In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA


llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.run("Where does Winstom live?")

'Winston lives in a place called Victory Mansions.'

In [2]:
chain.run("Describe Victory Mansions")

'Victory Mansions is described as a building with a hallway that smells of boiled cabbage and old rag mats. It has glass doors that lead into the interior, and the lift is seldom working, forcing residents to climb several flights of stairs. The building is characterized by its grimy appearance, and on the walls, there are large posters depicting the face of a man with a heavy black moustache, captioned with "BIG BROTHER IS WATCHING YOU." Overall, it conveys a sense of decay and oppressive atmosphere typical of the society in which Winston Smith lives.'

In [3]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

print(chain.run("Describe Victory Mansions"))

Victory Mansions is portrayed as a dilapidated apartment building where Winston Smith resides in George Orwell's "1984." The building is characterized by its grim and neglected environment, with an entrance featuring glass doors that allow dust to swirl in, emphasizing the decay of the setting. The hallway emits an unpleasant odor of boiled cabbage, further reflecting the neglect and drabness of life under the oppressive regime.

Winston's flat is notable for its unusual layout, particularly the positioning of the telescreen, which is situated on a longer wall opposite the window rather than the typical end wall. This arrangement allows Winston to sit in a shallow alcove, enabling him to remain outside the telescreen's line of sight, although he knows he can still be heard. The presence of the telescreen, a device that broadcasts Party propaganda and intrudes on his privacy, illustrates the pervasive surveillance that dominates his life.

In this oppressive environment, Winston finds s